# Imports

In [1]:
import importlib.metadata
import pandas as pd
import warnings
from datetime import datetime
from pathlib import Path

from diive.core.io.filereader import search_files, MultiDataFileReader
from diive.core.io.files import save_parquet, load_parquet
# pd.options.display.width = None
# pd.options.display.max_columns = None
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 3000)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)
dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
version_diive = importlib.metadata.version("diive")
print(f"diive version: v{version_diive}")

diive version: v0.85.0


# Load data: fluxes L1, meteo (10 variables) and management

In [2]:
irga = load_parquet(filepath="31.1_CH-CHA_IRGA_Level-1_eddypro_fluxnet_2005-2024.parquet")
qcllgr = load_parquet(filepath="32.1_CH-CHA_QCL+LGR_Level-1_eddypro_fluxnet_2012-2022.parquet")
meteo10 = load_parquet(filepath=Path("../10_METEO/") / "17.5_CH-CHA_meteo10_2005-2024.parquet")
mgmt = pd.read_csv(Path("../20_MANAGEMENT/") / "22.5_mgmt_full_timestamp.csv")
mgmt = mgmt.set_index('TIMESTAMP_MIDDLE')
mgmt.index = pd.to_datetime(mgmt.index)

Loaded .parquet file 31.1_CH-CHA_IRGA_Level-1_eddypro_fluxnet_2005-2024.parquet (0.612 seconds).
    --> Detected time resolution of <30 * Minutes> / 30min 
Loaded .parquet file 32.1_CH-CHA_QCL+LGR_Level-1_eddypro_fluxnet_2012-2022.parquet (1.396 seconds).
    --> Detected time resolution of <30 * Minutes> / 30min 
Loaded .parquet file ..\10_METEO\17.5_CH-CHA_meteo10_2005-2024.parquet (0.108 seconds).
    --> Detected time resolution of <30 * Minutes> / 30min 


# Merge files

## Start with IRGA data

In [3]:
df = irga.copy()
df

,AIR_MV,AIR_DENSITY,AIR_RHO_CP,AIR_CP,AOA_METHOD,AXES_ROTATION_METHOD,BOWEN,BURBA_METHOD,BADM_LOCATION_LAT,BADM_LOCATION_LONG,BADM_LOCATION_ELEV,BADM_HEIGHTC,BADM_INST_SAMPLING_INT,BADM_INST_AVERAGING_INT,BADM_INST_HEIGHT_SA,BADM_SA_OFFSET_NORTH,BADM_INSTPAIR_NORTHWARD_SEP_GA_CO2,BADM_INSTPAIR_EASTWARD_SEP_GA_CO2,BADM_INSTPAIR_HEIGHT_SEP_GA_CO2,BADM_INSTPAIR_NORTHWARD_SEP_GA_H2O,BADM_INSTPAIR_EASTWARD_SEP_GA_H2O,BADM_INSTPAIR_HEIGHT_SEP_GA_H2O,CUSTOM_FILTER_NR,CO2_NR,CO2_MEAS_TYPE,CO2_MOLAR_DENSITY,CO2_MIXING_RATIO,CO2,CO2_TLAG_ACTUAL,CO2_TLAG_USED,CO2_TLAG_NOMINAL,CO2_TLAG_MIN,CO2_TLAG_MAX,CO2_MEAS_MEDIAN,CO2_MEAS_P25,CO2_MEAS_P75,CO2_MEAS_SIGMA,CO2_MEAS_SKW,CO2_MEAS_KUR,CO2_MEAS_H2O_MEAS_COV,CO2_NUM_SPIKES,CUSTOM_FILTER_NREX,CO2_DIAG_NREX,CO2_SPIKE_NREX,CO2_ABSLIM_NREX,CO2_VM97_TEST,CO2_LGD,CO2_KID,CO2_ZCD,CUSTOM_DATA_SIZE_IRGA75_MEAN,CUSTOM_STATUS_CODE_IRGA75_MEAN,CUSTOM_GA_DIAG_CODE_IRGA75_MEAN,CUSTOM_AGC_MEAN,CUSTOM_FAST_T_MEAN,CUSTOM_AIR_P_MEAN,CUSTOM_COOLER_V_MEAN,DOY_START,DOY_END,DRYAIR_PARTIAL_PRESSURE,DRYAIR_DENSITY,DRYAIR_MV,DETRENDING_METHOD,DENTRENDING_TIME_CONSTANT,DISPLACEMENT_HEIGHT,EXPECT_NR,ET,ET_RANDUNC_HF,ET_UNCORR,ET_STAGE1,ET_STAGE2,ET_SCF,ET_CORRDIFF,ET_SSITC_TEST,FILE_NR,FC_NR,FC,FH2O,FC_RANDUNC_HF,FH2O_RANDUNC_HF,FC_VADV,FH2O_VADV,FETCH_MAX,FETCH_OFFSET,FETCH_10,FETCH_30,FETCH_50,FETCH_70,FETCH_80,FETCH_90,FC_UNCORR,FH2O_UNCORR,FC_STAGE1,FH2O_STAGE1,FC_STAGE2,FH2O_STAGE2,FC_SCF,FH2O_SCF,FC_CORRDIFF,FH2O_CORRDIFF,FC_NSR,FH2O_NSR,FC_SS,FH2O_SS,FC_SS_TEST,FH2O_SS_TEST,FC_SSITC_TEST,FH2O_SSITC_TEST,FOOTPRINT_MODEL,FILE_TIME_DURATION,H2O_NR,H_NR,H,H_RANDUNC_HF,H2O_MEAS_TYPE,H2O_MOLAR_DENSITY,H2O_MIXING_RATIO,H2O,H2O_TLAG_ACTUAL,H2O_TLAG_USED,H2O_TLAG_NOMINAL,H2O_TLAG_MIN,H2O_TLAG_MAX,H2O_MEAS_MEDIAN,H2O_MEAS_P25,H2O_MEAS_P75,H2O_MEAS_SIGMA,H2O_MEAS_SKW,H2O_MEAS_KUR,H_UNCORR,H_STAGE1,H_STAGE2,H_SCF,H2O_NUM_SPIKES,H2O_DIAG_NREX,H2O_SPIKE_NREX,H2O_ABSLIM_NREX,H2O_VM97_TEST,H2O_LGD,H2O_KID,H2O_ZCD,H_CORRDIFF,H_NSR,H_SS,H_SS_TEST,H_SSITC_TEST,HPATH_SA,HPATH_GA_CO2,HPATH_GA_H2O,INST_LI7200_AGC_OR_RSSI,INST_LI7500_AGC_OR_RSSI,LE_NR,LE,LE_RANDUNC_HF,LE_UNCORR,LE_STAGE1,LE_STAGE2,LE_SCF,LE_CORRDIFF,LE_SSITC_TEST,MO_LENGTH,MO_LENGTH_UNCORR,NIGHT,NUM_CUSTOM_VARS,NUM_BIOMET_VARS,PA_EP,PA_CELL,RH_EP,ROT_YAW,ROT_PITCH,ROUGHNESS_LENGTH,RESPONSE_TIME_SA,RESPONSE_TIME_GA_CO2,RESPONSE_TIME_GA_H2O,SW_IN_POT,SONIC_NR,SH_SINGLE,SLE_SINGLE,SET_SINGLE,SC_SINGLE,SH2O_SINGLE,SPECIFIC_HUMIDITY,SPECIFIC_HEAT_EVAP,SONIC_DIAG_NREX,SPECTRAL_CORRECTION_METHOD,TIMESTAMP_START,T_SONIC_NR,TAU_NR,TAU,TAU_RANDUNC_HF,TKE,TSTAR,T_SONIC,TA_EP,TDEW,T_SONIC_MEDIAN,T_SONIC_P25,T_SONIC_P75,T_SONIC_SIGMA,T_SONIC_SKW,T_SONIC_KUR,TAU_UNCORR,TAU_STAGE1,TAU_STAGE2,T_CELL,TAU_SCF,T_SONIC_NUM_SPIKES,T_SONIC_SPIKE_NREX,T_SONIC_ABSLIM_NREX,T_SONIC_VM97_TEST,T_SONIC_LGD,T_SONIC_KID,T_SONIC_ZCD,TAU_CORRDIFF,TAU_NSR,TAU_SS,T_SONIC_ITC,TAU_SS_TEST,T_SONIC_ITC_TEST,TAU_SSITC_TEST,TIMELAG_DETECTION_METHOD,U_UNROT,U,USTAR,U_MEDIAN,U_P25,U_P75,U_SIGMA,U_SKW,U_KUR,USTAR_UNCORR,U_NUM_SPIKES,U_SPIKE_NREX,U_ABSLIM_NREX,U_VM97_TEST,U_LGD,U_KID,U_ZCD,U_ITC,U_ITC_TEST,V_UNROT,V,VAPOR_DENSITY,VAPOR_PARTIAL_PRESSURE,VAPOR_PARTIAL_PRESSURE_SAT,VPD_EP,VAPOR_DRYAIR_RATIO,V_MEDIAN,V_P25,V_P75,V_SIGMA,V_SKW,V_KUR,V_NUM_SPIKES,V_SPIKE_NREX,V_ABSLIM_NREX,V_VM97_TEST,VM97_TLAG_HF,VM97_TLAG_SF,VM97_AOA_HF,VM97_NSHW_HF,V_LGD,V_KID,V_ZCD,VPATH_SA,VPATH_GA_CO2,VPATH_GA_H2O,WD_FILTER_NR,W_UNROT,W,WS,WS_MAX,WD,WD_SIGMA,W_MEDIAN,W_P25,W_P75,W_SIGMA,W_SKW,W_KUR,W_U_COV,W_T_SONIC_COV,W_CO2_MEAS_COV,W_H2O_MEAS_COV,W_T_SONIC_COV_IBROM,W_T_SONIC_COV_IBROM_N1626,W_T_SONIC_COV_IBROM_N0614,W_T_SONIC_COV_IBROM_N0277,W_T_SONIC_COV_IBROM_N0133,W_T_SONIC_COV_IBROM_N0065,W_T_SONIC_COV_IBROM_N0032,W_T_SONIC_COV_IBROM_N0016,W_T_SONIC_COV_IBROM_N0008,W_T_SONIC_COV_IBROM_N0004,W_NUM_SPIKES,WD_FILTER_NREX,W_SPIKE_NREX,W_ABSLIM_NREX,W_VM97_TEST,W_LGD,W_KID,W_ZCD,W_ITC,W_ITC_TEST,WBOOST_APPLIED,WPL_APPLIED,ZL,ZL_UNCORR
TIMESTAMP_MIDDLE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

## Add *new* columns from QCL LGR data

In [4]:
newcols = [c for c in qcllgr.columns if c not in df.columns]
df = pd.concat([df, qcllgr[newcols]], axis=1)
df

,AIR_MV,AIR_DENSITY,AIR_RHO_CP,AIR_CP,AOA_METHOD,AXES_ROTATION_METHOD,BOWEN,BURBA_METHOD,BADM_LOCATION_LAT,BADM_LOCATION_LONG,BADM_LOCATION_ELEV,BADM_HEIGHTC,BADM_INST_SAMPLING_INT,BADM_INST_AVERAGING_INT,BADM_INST_HEIGHT_SA,BADM_SA_OFFSET_NORTH,BADM_INSTPAIR_NORTHWARD_SEP_GA_CO2,BADM_INSTPAIR_EASTWARD_SEP_GA_CO2,BADM_INSTPAIR_HEIGHT_SEP_GA_CO2,BADM_INSTPAIR_NORTHWARD_SEP_GA_H2O,BADM_INSTPAIR_EASTWARD_SEP_GA_H2O,BADM_INSTPAIR_HEIGHT_SEP_GA_H2O,CUSTOM_FILTER_NR,CO2_NR,CO2_MEAS_TYPE,CO2_MOLAR_DENSITY,CO2_MIXING_RATIO,CO2,CO2_TLAG_ACTUAL,CO2_TLAG_USED,CO2_TLAG_NOMINAL,CO2_TLAG_MIN,CO2_TLAG_MAX,CO2_MEAS_MEDIAN,CO2_MEAS_P25,CO2_MEAS_P75,CO2_MEAS_SIGMA,CO2_MEAS_SKW,CO2_MEAS_KUR,CO2_MEAS_H2O_MEAS_COV,CO2_NUM_SPIKES,CUSTOM_FILTER_NREX,CO2_DIAG_NREX,CO2_SPIKE_NREX,CO2_ABSLIM_NREX,CO2_VM97_TEST,CO2_LGD,CO2_KID,CO2_ZCD,CUSTOM_DATA_SIZE_IRGA75_MEAN,CUSTOM_STATUS_CODE_IRGA75_MEAN,CUSTOM_GA_DIAG_CODE_IRGA75_MEAN,CUSTOM_AGC_MEAN,CUSTOM_FAST_T_MEAN,CUSTOM_AIR_P_MEAN,CUSTOM_COOLER_V_MEAN,DOY_START,DOY_END,DRYAIR_PARTIAL_PRESSURE,DRYAIR_DENSITY,DRYAIR_MV,DETRENDING_METHOD,DENTRENDING_TIME_CONSTANT,DISPLACEMENT_HEIGHT,EXPECT_NR,ET,ET_RANDUNC_HF,ET_UNCORR,ET_STAGE1,ET_STAGE2,ET_SCF,ET_CORRDIFF,ET_SSITC_TEST,FILE_NR,FC_NR,FC,FH2O,FC_RANDUNC_HF,FH2O_RANDUNC_HF,FC_VADV,FH2O_VADV,FETCH_MAX,FETCH_OFFSET,FETCH_10,FETCH_30,FETCH_50,FETCH_70,FETCH_80,FETCH_90,FC_UNCORR,FH2O_UNCORR,FC_STAGE1,FH2O_STAGE1,FC_STAGE2,FH2O_STAGE2,FC_SCF,FH2O_SCF,FC_CORRDIFF,FH2O_CORRDIFF,FC_NSR,FH2O_NSR,FC_SS,FH2O_SS,FC_SS_TEST,FH2O_SS_TEST,FC_SSITC_TEST,FH2O_SSITC_TEST,FOOTPRINT_MODEL,FILE_TIME_DURATION,H2O_NR,H_NR,H,H_RANDUNC_HF,H2O_MEAS_TYPE,H2O_MOLAR_DENSITY,H2O_MIXING_RATIO,H2O,H2O_TLAG_ACTUAL,H2O_TLAG_USED,H2O_TLAG_NOMINAL,H2O_TLAG_MIN,H2O_TLAG_MAX,H2O_MEAS_MEDIAN,H2O_MEAS_P25,H2O_MEAS_P75,H2O_MEAS_SIGMA,H2O_MEAS_SKW,H2O_MEAS_KUR,H_UNCORR,H_STAGE1,H_STAGE2,H_SCF,H2O_NUM_SPIKES,H2O_DIAG_NREX,H2O_SPIKE_NREX,H2O_ABSLIM_NREX,H2O_VM97_TEST,H2O_LGD,H2O_KID,H2O_ZCD,H_CORRDIFF,H_NSR,H_SS,H_SS_TEST,H_SSITC_TEST,HPATH_SA,HPATH_GA_CO2,HPATH_GA_H2O,INST_LI7200_AGC_OR_RSSI,INST_LI7500_AGC_OR_RSSI,LE_NR,LE,LE_RANDUNC_HF,LE_UNCORR,LE_STAGE1,LE_STAGE2,LE_SCF,LE_CORRDIFF,LE_SSITC_TEST,MO_LENGTH,MO_LENGTH_UNCORR,NIGHT,NUM_CUSTOM_VARS,NUM_BIOMET_VARS,PA_EP,PA_CELL,RH_EP,ROT_YAW,ROT_PITCH,ROUGHNESS_LENGTH,RESPONSE_TIME_SA,RESPONSE_TIME_GA_CO2,RESPONSE_TIME_GA_H2O,SW_IN_POT,SONIC_NR,SH_SINGLE,SLE_SINGLE,SET_SINGLE,SC_SINGLE,SH2O_SINGLE,SPECIFIC_HUMIDITY,SPECIFIC_HEAT_EVAP,SONIC_DIAG_NREX,SPECTRAL_CORRECTION_METHOD,TIMESTAMP_START,T_SONIC_NR,TAU_NR,TAU,TAU_RANDUNC_HF,TKE,TSTAR,T_SONIC,TA_EP,TDEW,T_SONIC_MEDIAN,T_SONIC_P25,T_SONIC_P75,T_SONIC_SIGMA,T_SONIC_SKW,T_SONIC_KUR,TAU_UNCORR,TAU_STAGE1,TAU_STAGE2,T_CELL,TAU_SCF,T_SONIC_NUM_SPIKES,T_SONIC_SPIKE_NREX,T_SONIC_ABSLIM_NREX,T_SONIC_VM97_TEST,T_SONIC_LGD,T_SONIC_KID,T_SONIC_ZCD,TAU_CORRDIFF,TAU_NSR,TAU_SS,T_SONIC_ITC,TAU_SS_TEST,T_SONIC_ITC_TEST,TAU_SSITC_TEST,TIMELAG_DETECTION_METHOD,U_UNROT,U,USTAR,U_MEDIAN,U_P25,U_P75,U_SIGMA,U_SKW,U_KUR,USTAR_UNCORR,U_NUM_SPIKES,U_SPIKE_NREX,U_ABSLIM_NREX,U_VM97_TEST,U_LGD,U_KID,U_ZCD,U_ITC,U_ITC_TEST,V_UNROT,V,VAPOR_DENSITY,VAPOR_PARTIAL_PRESSURE,VAPOR_PARTIAL_PRESSURE_SAT,VPD_EP,VAPOR_DRYAIR_RATIO,V_MEDIAN,V_P25,V_P75,V_SIGMA,V_SKW,V_KUR,V_NUM_SPIKES,V_SPIKE_NREX,V_ABSLIM_NREX,V_VM97_TEST,VM97_TLAG_HF,VM97_TLAG_SF,VM97_AOA_HF,VM97_NSHW_HF,V_LGD,V_KID,V_ZCD,VPATH_SA,VPATH_GA_CO2,VPATH_GA_H2O,WD_FILTER_NR,W_UNROT,W,WS,WS_MAX,WD,WD_SIGMA,W_MEDIAN,W_P25,W_P75,W_SIGMA,W_SKW,W_KUR,W_U_COV,W_T_SONIC_COV,W_CO2_MEAS_COV,W_H2O_MEAS_COV,W_T_SONIC_COV_IBROM,W_T_SONIC_COV_IBROM_N1626,W_T_SONIC_COV_IBROM_N0614,W_T_SONIC_COV_IBROM_N0277,W_T_SONIC_COV_IBROM_N0133,W_T_SONIC_COV_IBROM_N0065,W_T_SONIC_COV_IBROM_N0032,W_T_SONIC_COV_IBROM_N0016,W_T_SONIC_COV_IBROM_N0008,W_T_SONIC_COV_IBROM_N0004,W_NUM_SPIKES,WD_FILTER_NREX,W_SPIKE_NREX,W_ABSLIM_NREX,W_VM97_TEST,W_LGD,W_KID,W_ZCD,W_ITC,W_ITC_TEST,WBOOST_APPLIED,WPL_APPLIED,ZL,ZL_UNCORR,BADM_INSTPAIR_EASTWARD_SEP_GA_CH4,BADM_INSTPAIR_EASTWARD_SEP_GA_N2O,BADM_INSTPAIR_HEIGHT_S

## Add *new* columns from meteo data

In [5]:
newcols = [c for c in meteo10.columns if c not in df.columns]
df = pd.concat([df, meteo10[newcols]], axis=1)
display(newcols)
df

['FLAG_PA_GF1_0.9_1_ISFILLED',
 'FLAG_PPFD_IN_T1_2_2_ISFILLED',
 'FLAG_PREC_RAIN_TOT_GF1_0.5_1_FLUXNET_ISFILLED',
 'FLAG_PREC_RAIN_TOT_GF1_0.5_1_ISFILLED',
 'FLAG_RH_T1_2_1_ISFILLED',
 'FLAG_SWC_GF1_0.15_1_gfXG_ISFILLED',
 'FLAG_SW_IN_T1_2_1_ISFILLED',
 'FLAG_TA_T1_2_1_ISFILLED',
 'FLAG_TS_GF1_0.04_1_gfXG_ISFILLED',
 'FLAG_TS_GF1_0.15_1_gfXG_ISFILLED',
 'FLAG_TS_GF1_0.4_1_gfXG_ISFILLED',
 'FLAG_VPD_T1_2_1_ISFILLED',
 'LW_IN_T1_2_1',
 'PA_GF1_0.9_1',
 'PPFD_IN_T1_2_2',
 'PREC_RAIN_TOT_GF1_0.5_1',
 'RH_T1_2_1',
 'SWC_GF1_0.05_1',
 'SWC_GF1_0.15_1',
 'SWC_GF1_0.15_1_gfXG',
 'SWC_GF1_0.75_1',
 'SW_IN_T1_2_1',
 'TA_T1_2_1',
 'TIMESINCE_PREC_RAIN_TOT_GF1_0.5_1',
 'TS_GF1_0.04_1',
 'TS_GF1_0.04_1_gfXG',
 'TS_GF1_0.15_1',
 'TS_GF1_0.15_1_gfXG',
 'TS_GF1_0.4_1',
 'TS_GF1_0.4_1_gfXG',
 'VPD_T1_2_1']

,AIR_MV,AIR_DENSITY,AIR_RHO_CP,AIR_CP,AOA_METHOD,AXES_ROTATION_METHOD,BOWEN,BURBA_METHOD,BADM_LOCATION_LAT,BADM_LOCATION_LONG,BADM_LOCATION_ELEV,BADM_HEIGHTC,BADM_INST_SAMPLING_INT,BADM_INST_AVERAGING_INT,BADM_INST_HEIGHT_SA,BADM_SA_OFFSET_NORTH,BADM_INSTPAIR_NORTHWARD_SEP_GA_CO2,BADM_INSTPAIR_EASTWARD_SEP_GA_CO2,BADM_INSTPAIR_HEIGHT_SEP_GA_CO2,BADM_INSTPAIR_NORTHWARD_SEP_GA_H2O,BADM_INSTPAIR_EASTWARD_SEP_GA_H2O,BADM_INSTPAIR_HEIGHT_SEP_GA_H2O,CUSTOM_FILTER_NR,CO2_NR,CO2_MEAS_TYPE,CO2_MOLAR_DENSITY,CO2_MIXING_RATIO,CO2,CO2_TLAG_ACTUAL,CO2_TLAG_USED,CO2_TLAG_NOMINAL,CO2_TLAG_MIN,CO2_TLAG_MAX,CO2_MEAS_MEDIAN,CO2_MEAS_P25,CO2_MEAS_P75,CO2_MEAS_SIGMA,CO2_MEAS_SKW,CO2_MEAS_KUR,CO2_MEAS_H2O_MEAS_COV,CO2_NUM_SPIKES,CUSTOM_FILTER_NREX,CO2_DIAG_NREX,CO2_SPIKE_NREX,CO2_ABSLIM_NREX,CO2_VM97_TEST,CO2_LGD,CO2_KID,CO2_ZCD,CUSTOM_DATA_SIZE_IRGA75_MEAN,CUSTOM_STATUS_CODE_IRGA75_MEAN,CUSTOM_GA_DIAG_CODE_IRGA75_MEAN,CUSTOM_AGC_MEAN,CUSTOM_FAST_T_MEAN,CUSTOM_AIR_P_MEAN,CUSTOM_COOLER_V_MEAN,DOY_START,DOY_END,DRYAIR_PARTIAL_PRESSURE,DRYAIR_DENSITY,DRYAIR_MV,DETRENDING_METHOD,DENTRENDING_TIME_CONSTANT,DISPLACEMENT_HEIGHT,EXPECT_NR,ET,ET_RANDUNC_HF,ET_UNCORR,ET_STAGE1,ET_STAGE2,ET_SCF,ET_CORRDIFF,ET_SSITC_TEST,FILE_NR,FC_NR,FC,FH2O,FC_RANDUNC_HF,FH2O_RANDUNC_HF,FC_VADV,FH2O_VADV,FETCH_MAX,FETCH_OFFSET,FETCH_10,FETCH_30,FETCH_50,FETCH_70,FETCH_80,FETCH_90,FC_UNCORR,FH2O_UNCORR,FC_STAGE1,FH2O_STAGE1,FC_STAGE2,FH2O_STAGE2,FC_SCF,FH2O_SCF,FC_CORRDIFF,FH2O_CORRDIFF,FC_NSR,FH2O_NSR,FC_SS,FH2O_SS,FC_SS_TEST,FH2O_SS_TEST,FC_SSITC_TEST,FH2O_SSITC_TEST,FOOTPRINT_MODEL,FILE_TIME_DURATION,H2O_NR,H_NR,H,H_RANDUNC_HF,H2O_MEAS_TYPE,H2O_MOLAR_DENSITY,H2O_MIXING_RATIO,H2O,H2O_TLAG_ACTUAL,H2O_TLAG_USED,H2O_TLAG_NOMINAL,H2O_TLAG_MIN,H2O_TLAG_MAX,H2O_MEAS_MEDIAN,H2O_MEAS_P25,H2O_MEAS_P75,H2O_MEAS_SIGMA,H2O_MEAS_SKW,H2O_MEAS_KUR,H_UNCORR,H_STAGE1,H_STAGE2,H_SCF,H2O_NUM_SPIKES,H2O_DIAG_NREX,H2O_SPIKE_NREX,H2O_ABSLIM_NREX,H2O_VM97_TEST,H2O_LGD,H2O_KID,H2O_ZCD,H_CORRDIFF,H_NSR,H_SS,H_SS_TEST,H_SSITC_TEST,HPATH_SA,HPATH_GA_CO2,HPATH_GA_H2O,INST_LI7200_AGC_OR_RSSI,INST_LI7500_AGC_OR_RSSI,LE_NR,LE,LE_RANDUNC_HF,LE_UNCORR,LE_STAGE1,LE_STAGE2,LE_SCF,LE_CORRDIFF,LE_SSITC_TEST,MO_LENGTH,MO_LENGTH_UNCORR,NIGHT,NUM_CUSTOM_VARS,NUM_BIOMET_VARS,PA_EP,PA_CELL,RH_EP,ROT_YAW,ROT_PITCH,ROUGHNESS_LENGTH,RESPONSE_TIME_SA,RESPONSE_TIME_GA_CO2,RESPONSE_TIME_GA_H2O,SW_IN_POT,SONIC_NR,SH_SINGLE,SLE_SINGLE,SET_SINGLE,SC_SINGLE,SH2O_SINGLE,SPECIFIC_HUMIDITY,SPECIFIC_HEAT_EVAP,SONIC_DIAG_NREX,SPECTRAL_CORRECTION_METHOD,TIMESTAMP_START,T_SONIC_NR,TAU_NR,TAU,TAU_RANDUNC_HF,TKE,TSTAR,T_SONIC,TA_EP,TDEW,T_SONIC_MEDIAN,T_SONIC_P25,T_SONIC_P75,T_SONIC_SIGMA,T_SONIC_SKW,T_SONIC_KUR,TAU_UNCORR,TAU_STAGE1,TAU_STAGE2,T_CELL,TAU_SCF,T_SONIC_NUM_SPIKES,T_SONIC_SPIKE_NREX,T_SONIC_ABSLIM_NREX,T_SONIC_VM97_TEST,T_SONIC_LGD,T_SONIC_KID,T_SONIC_ZCD,TAU_CORRDIFF,TAU_NSR,TAU_SS,T_SONIC_ITC,TAU_SS_TEST,T_SONIC_ITC_TEST,TAU_SSITC_TEST,TIMELAG_DETECTION_METHOD,U_UNROT,U,USTAR,U_MEDIAN,U_P25,U_P75,U_SIGMA,U_SKW,U_KUR,USTAR_UNCORR,U_NUM_SPIKES,U_SPIKE_NREX,U_ABSLIM_NREX,U_VM97_TEST,U_LGD,U_KID,U_ZCD,U_ITC,U_ITC_TEST,V_UNROT,V,VAPOR_DENSITY,VAPOR_PARTIAL_PRESSURE,VAPOR_PARTIAL_PRESSURE_SAT,VPD_EP,VAPOR_DRYAIR_RATIO,V_MEDIAN,V_P25,V_P75,V_SIGMA,V_SKW,V_KUR,V_NUM_SPIKES,V_SPIKE_NREX,V_ABSLIM_NREX,V_VM97_TEST,VM97_TLAG_HF,VM97_TLAG_SF,VM97_AOA_HF,VM97_NSHW_HF,V_LGD,V_KID,V_ZCD,VPATH_SA,VPATH_GA_CO2,VPATH_GA_H2O,WD_FILTER_NR,W_UNROT,W,WS,WS_MAX,WD,WD_SIGMA,W_MEDIAN,W_P25,W_P75,W_SIGMA,W_SKW,W_KUR,W_U_COV,W_T_SONIC_COV,W_CO2_MEAS_COV,W_H2O_MEAS_COV,W_T_SONIC_COV_IBROM,W_T_SONIC_COV_IBROM_N1626,W_T_SONIC_COV_IBROM_N0614,W_T_SONIC_COV_IBROM_N0277,W_T_SONIC_COV_IBROM_N0133,W_T_SONIC_COV_IBROM_N0065,W_T_SONIC_COV_IBROM_N0032,W_T_SONIC_COV_IBROM_N0016,W_T_SONIC_COV_IBROM_N0008,W_T_SONIC_COV_IBROM_N0004,W_NUM_SPIKES,WD_FILTER_NREX,W_SPIKE_NREX,W_ABSLIM_NREX,W_VM97_TEST,W_LGD,W_KID,W_ZCD,W_ITC,W_ITC_TEST,WBOOST_APPLIED,WPL_APPLIED,ZL,ZL_UNCORR,BADM_INSTPAIR_EASTWARD_SEP_GA_CH4,BADM_INSTPAIR_EASTWARD_SEP_GA_N2O,BADM_INSTPAIR_HEIGHT_S

## Add *new* columns from management data

In [6]:
newcols = [c for c in mgmt.columns if c not in df.columns]
df = pd.concat([df, mgmt[newcols]], axis=1)
display(newcols)
df

['MGMT_FERT_MIN',
 'MGMT_FERT_MIN_FOOTPRINT',
 'MGMT_FERT_MIN_PARCEL-A',
 'MGMT_FERT_MIN_PARCEL-B',
 'MGMT_FERT_ORG',
 'MGMT_FERT_ORG_FOOTPRINT',
 'MGMT_FERT_ORG_PARCEL-A',
 'MGMT_FERT_ORG_PARCEL-B',
 'MGMT_GRAZING',
 'MGMT_GRAZING_FOOTPRINT',
 'MGMT_GRAZING_PARCEL-A',
 'MGMT_GRAZING_PARCEL-B',
 'MGMT_MOWING',
 'MGMT_MOWING_FOOTPRINT',
 'MGMT_MOWING_PARCEL-A',
 'MGMT_MOWING_PARCEL-B',
 'MGMT_PESTICIDE_HERBICIDE',
 'MGMT_PESTICIDE_HERBICIDE_FOOTPRINT',
 'MGMT_PESTICIDE_HERBICIDE_PARCEL-A',
 'MGMT_PESTICIDE_HERBICIDE_PARCEL-B',
 'MGMT_SOILCULTIVATION',
 'MGMT_SOILCULTIVATION_FOOTPRINT',
 'MGMT_SOILCULTIVATION_PARCEL-A',
 'MGMT_SOILCULTIVATION_PARCEL-B',
 'MGMT_SOWING',
 'MGMT_SOWING_FOOTPRINT',
 'MGMT_SOWING_PARCEL-A',
 'MGMT_SOWING_PARCEL-B',
 'TIMESINCE_MGMT_FERT_MIN',
 'TIMESINCE_MGMT_FERT_MIN_FOOTPRINT',
 'TIMESINCE_MGMT_FERT_MIN_PARCEL-A',
 'TIMESINCE_MGMT_FERT_MIN_PARCEL-B',
 'TIMESINCE_MGMT_FERT_ORG',
 'TIMESINCE_MGMT_FERT_ORG_FOOTPRINT',
 'TIMESINCE_MGMT_FERT_ORG_PARCEL-A',
 'TIM

,AIR_MV,AIR_DENSITY,AIR_RHO_CP,AIR_CP,AOA_METHOD,AXES_ROTATION_METHOD,BOWEN,BURBA_METHOD,BADM_LOCATION_LAT,BADM_LOCATION_LONG,BADM_LOCATION_ELEV,BADM_HEIGHTC,BADM_INST_SAMPLING_INT,BADM_INST_AVERAGING_INT,BADM_INST_HEIGHT_SA,BADM_SA_OFFSET_NORTH,BADM_INSTPAIR_NORTHWARD_SEP_GA_CO2,BADM_INSTPAIR_EASTWARD_SEP_GA_CO2,BADM_INSTPAIR_HEIGHT_SEP_GA_CO2,BADM_INSTPAIR_NORTHWARD_SEP_GA_H2O,BADM_INSTPAIR_EASTWARD_SEP_GA_H2O,BADM_INSTPAIR_HEIGHT_SEP_GA_H2O,CUSTOM_FILTER_NR,CO2_NR,CO2_MEAS_TYPE,CO2_MOLAR_DENSITY,CO2_MIXING_RATIO,CO2,CO2_TLAG_ACTUAL,CO2_TLAG_USED,CO2_TLAG_NOMINAL,CO2_TLAG_MIN,CO2_TLAG_MAX,CO2_MEAS_MEDIAN,CO2_MEAS_P25,CO2_MEAS_P75,CO2_MEAS_SIGMA,CO2_MEAS_SKW,CO2_MEAS_KUR,CO2_MEAS_H2O_MEAS_COV,CO2_NUM_SPIKES,CUSTOM_FILTER_NREX,CO2_DIAG_NREX,CO2_SPIKE_NREX,CO2_ABSLIM_NREX,CO2_VM97_TEST,CO2_LGD,CO2_KID,CO2_ZCD,CUSTOM_DATA_SIZE_IRGA75_MEAN,CUSTOM_STATUS_CODE_IRGA75_MEAN,CUSTOM_GA_DIAG_CODE_IRGA75_MEAN,CUSTOM_AGC_MEAN,CUSTOM_FAST_T_MEAN,CUSTOM_AIR_P_MEAN,CUSTOM_COOLER_V_MEAN,DOY_START,DOY_END,DRYAIR_PARTIAL_PRESSURE,DRYAIR_DENSITY,DRYAIR_MV,DETRENDING_METHOD,DENTRENDING_TIME_CONSTANT,DISPLACEMENT_HEIGHT,EXPECT_NR,ET,ET_RANDUNC_HF,ET_UNCORR,ET_STAGE1,ET_STAGE2,ET_SCF,ET_CORRDIFF,ET_SSITC_TEST,FILE_NR,FC_NR,FC,FH2O,FC_RANDUNC_HF,FH2O_RANDUNC_HF,FC_VADV,FH2O_VADV,FETCH_MAX,FETCH_OFFSET,FETCH_10,FETCH_30,FETCH_50,FETCH_70,FETCH_80,FETCH_90,FC_UNCORR,FH2O_UNCORR,FC_STAGE1,FH2O_STAGE1,FC_STAGE2,FH2O_STAGE2,FC_SCF,FH2O_SCF,FC_CORRDIFF,FH2O_CORRDIFF,FC_NSR,FH2O_NSR,FC_SS,FH2O_SS,FC_SS_TEST,FH2O_SS_TEST,FC_SSITC_TEST,FH2O_SSITC_TEST,FOOTPRINT_MODEL,FILE_TIME_DURATION,H2O_NR,H_NR,H,H_RANDUNC_HF,H2O_MEAS_TYPE,H2O_MOLAR_DENSITY,H2O_MIXING_RATIO,H2O,H2O_TLAG_ACTUAL,H2O_TLAG_USED,H2O_TLAG_NOMINAL,H2O_TLAG_MIN,H2O_TLAG_MAX,H2O_MEAS_MEDIAN,H2O_MEAS_P25,H2O_MEAS_P75,H2O_MEAS_SIGMA,H2O_MEAS_SKW,H2O_MEAS_KUR,H_UNCORR,H_STAGE1,H_STAGE2,H_SCF,H2O_NUM_SPIKES,H2O_DIAG_NREX,H2O_SPIKE_NREX,H2O_ABSLIM_NREX,H2O_VM97_TEST,H2O_LGD,H2O_KID,H2O_ZCD,H_CORRDIFF,H_NSR,H_SS,H_SS_TEST,H_SSITC_TEST,HPATH_SA,HPATH_GA_CO2,HPATH_GA_H2O,INST_LI7200_AGC_OR_RSSI,INST_LI7500_AGC_OR_RSSI,LE_NR,LE,LE_RANDUNC_HF,LE_UNCORR,LE_STAGE1,LE_STAGE2,LE_SCF,LE_CORRDIFF,LE_SSITC_TEST,MO_LENGTH,MO_LENGTH_UNCORR,NIGHT,NUM_CUSTOM_VARS,NUM_BIOMET_VARS,PA_EP,PA_CELL,RH_EP,ROT_YAW,ROT_PITCH,ROUGHNESS_LENGTH,RESPONSE_TIME_SA,RESPONSE_TIME_GA_CO2,RESPONSE_TIME_GA_H2O,SW_IN_POT,SONIC_NR,SH_SINGLE,SLE_SINGLE,SET_SINGLE,SC_SINGLE,SH2O_SINGLE,SPECIFIC_HUMIDITY,SPECIFIC_HEAT_EVAP,SONIC_DIAG_NREX,SPECTRAL_CORRECTION_METHOD,TIMESTAMP_START,T_SONIC_NR,TAU_NR,TAU,TAU_RANDUNC_HF,TKE,TSTAR,T_SONIC,TA_EP,TDEW,T_SONIC_MEDIAN,T_SONIC_P25,T_SONIC_P75,T_SONIC_SIGMA,T_SONIC_SKW,T_SONIC_KUR,TAU_UNCORR,TAU_STAGE1,TAU_STAGE2,T_CELL,TAU_SCF,T_SONIC_NUM_SPIKES,T_SONIC_SPIKE_NREX,T_SONIC_ABSLIM_NREX,T_SONIC_VM97_TEST,T_SONIC_LGD,T_SONIC_KID,T_SONIC_ZCD,TAU_CORRDIFF,TAU_NSR,TAU_SS,T_SONIC_ITC,TAU_SS_TEST,T_SONIC_ITC_TEST,TAU_SSITC_TEST,TIMELAG_DETECTION_METHOD,U_UNROT,U,USTAR,U_MEDIAN,U_P25,U_P75,U_SIGMA,U_SKW,U_KUR,USTAR_UNCORR,U_NUM_SPIKES,U_SPIKE_NREX,U_ABSLIM_NREX,U_VM97_TEST,U_LGD,U_KID,U_ZCD,U_ITC,U_ITC_TEST,V_UNROT,V,VAPOR_DENSITY,VAPOR_PARTIAL_PRESSURE,VAPOR_PARTIAL_PRESSURE_SAT,VPD_EP,VAPOR_DRYAIR_RATIO,V_MEDIAN,V_P25,V_P75,V_SIGMA,V_SKW,V_KUR,V_NUM_SPIKES,V_SPIKE_NREX,V_ABSLIM_NREX,V_VM97_TEST,VM97_TLAG_HF,VM97_TLAG_SF,VM97_AOA_HF,VM97_NSHW_HF,V_LGD,V_KID,V_ZCD,VPATH_SA,VPATH_GA_CO2,VPATH_GA_H2O,WD_FILTER_NR,W_UNROT,W,WS,WS_MAX,WD,WD_SIGMA,W_MEDIAN,W_P25,W_P75,W_SIGMA,W_SKW,W_KUR,W_U_COV,W_T_SONIC_COV,W_CO2_MEAS_COV,W_H2O_MEAS_COV,W_T_SONIC_COV_IBROM,W_T_SONIC_COV_IBROM_N1626,W_T_SONIC_COV_IBROM_N0614,W_T_SONIC_COV_IBROM_N0277,W_T_SONIC_COV_IBROM_N0133,W_T_SONIC_COV_IBROM_N0065,W_T_SONIC_COV_IBROM_N0032,W_T_SONIC_COV_IBROM_N0016,W_T_SONIC_COV_IBROM_N0008,W_T_SONIC_COV_IBROM_N0004,W_NUM_SPIKES,WD_FILTER_NREX,W_SPIKE_NREX,W_ABSLIM_NREX,W_VM97_TEST,W_LGD,W_KID,W_ZCD,W_ITC,W_ITC_TEST,WBOOST_APPLIED,WPL_APPLIED,ZL,ZL_UNCORR,BADM_INSTPAIR_EASTWARD_SEP_GA_CH4,BADM_INSTPAIR_EASTWARD_SEP_GA_N2O,BADM_INSTPAIR_HEIGHT_S

## Restrict data time range

In [7]:
keep = (df.index.year >= 2005) & (df.index.year <= 2024)
df = df[keep].copy()
df

,AIR_MV,AIR_DENSITY,AIR_RHO_CP,AIR_CP,AOA_METHOD,AXES_ROTATION_METHOD,BOWEN,BURBA_METHOD,BADM_LOCATION_LAT,BADM_LOCATION_LONG,BADM_LOCATION_ELEV,BADM_HEIGHTC,BADM_INST_SAMPLING_INT,BADM_INST_AVERAGING_INT,BADM_INST_HEIGHT_SA,BADM_SA_OFFSET_NORTH,BADM_INSTPAIR_NORTHWARD_SEP_GA_CO2,BADM_INSTPAIR_EASTWARD_SEP_GA_CO2,BADM_INSTPAIR_HEIGHT_SEP_GA_CO2,BADM_INSTPAIR_NORTHWARD_SEP_GA_H2O,BADM_INSTPAIR_EASTWARD_SEP_GA_H2O,BADM_INSTPAIR_HEIGHT_SEP_GA_H2O,CUSTOM_FILTER_NR,CO2_NR,CO2_MEAS_TYPE,CO2_MOLAR_DENSITY,CO2_MIXING_RATIO,CO2,CO2_TLAG_ACTUAL,CO2_TLAG_USED,CO2_TLAG_NOMINAL,CO2_TLAG_MIN,CO2_TLAG_MAX,CO2_MEAS_MEDIAN,CO2_MEAS_P25,CO2_MEAS_P75,CO2_MEAS_SIGMA,CO2_MEAS_SKW,CO2_MEAS_KUR,CO2_MEAS_H2O_MEAS_COV,CO2_NUM_SPIKES,CUSTOM_FILTER_NREX,CO2_DIAG_NREX,CO2_SPIKE_NREX,CO2_ABSLIM_NREX,CO2_VM97_TEST,CO2_LGD,CO2_KID,CO2_ZCD,CUSTOM_DATA_SIZE_IRGA75_MEAN,CUSTOM_STATUS_CODE_IRGA75_MEAN,CUSTOM_GA_DIAG_CODE_IRGA75_MEAN,CUSTOM_AGC_MEAN,CUSTOM_FAST_T_MEAN,CUSTOM_AIR_P_MEAN,CUSTOM_COOLER_V_MEAN,DOY_START,DOY_END,DRYAIR_PARTIAL_PRESSURE,DRYAIR_DENSITY,DRYAIR_MV,DETRENDING_METHOD,DENTRENDING_TIME_CONSTANT,DISPLACEMENT_HEIGHT,EXPECT_NR,ET,ET_RANDUNC_HF,ET_UNCORR,ET_STAGE1,ET_STAGE2,ET_SCF,ET_CORRDIFF,ET_SSITC_TEST,FILE_NR,FC_NR,FC,FH2O,FC_RANDUNC_HF,FH2O_RANDUNC_HF,FC_VADV,FH2O_VADV,FETCH_MAX,FETCH_OFFSET,FETCH_10,FETCH_30,FETCH_50,FETCH_70,FETCH_80,FETCH_90,FC_UNCORR,FH2O_UNCORR,FC_STAGE1,FH2O_STAGE1,FC_STAGE2,FH2O_STAGE2,FC_SCF,FH2O_SCF,FC_CORRDIFF,FH2O_CORRDIFF,FC_NSR,FH2O_NSR,FC_SS,FH2O_SS,FC_SS_TEST,FH2O_SS_TEST,FC_SSITC_TEST,FH2O_SSITC_TEST,FOOTPRINT_MODEL,FILE_TIME_DURATION,H2O_NR,H_NR,H,H_RANDUNC_HF,H2O_MEAS_TYPE,H2O_MOLAR_DENSITY,H2O_MIXING_RATIO,H2O,H2O_TLAG_ACTUAL,H2O_TLAG_USED,H2O_TLAG_NOMINAL,H2O_TLAG_MIN,H2O_TLAG_MAX,H2O_MEAS_MEDIAN,H2O_MEAS_P25,H2O_MEAS_P75,H2O_MEAS_SIGMA,H2O_MEAS_SKW,H2O_MEAS_KUR,H_UNCORR,H_STAGE1,H_STAGE2,H_SCF,H2O_NUM_SPIKES,H2O_DIAG_NREX,H2O_SPIKE_NREX,H2O_ABSLIM_NREX,H2O_VM97_TEST,H2O_LGD,H2O_KID,H2O_ZCD,H_CORRDIFF,H_NSR,H_SS,H_SS_TEST,H_SSITC_TEST,HPATH_SA,HPATH_GA_CO2,HPATH_GA_H2O,INST_LI7200_AGC_OR_RSSI,INST_LI7500_AGC_OR_RSSI,LE_NR,LE,LE_RANDUNC_HF,LE_UNCORR,LE_STAGE1,LE_STAGE2,LE_SCF,LE_CORRDIFF,LE_SSITC_TEST,MO_LENGTH,MO_LENGTH_UNCORR,NIGHT,NUM_CUSTOM_VARS,NUM_BIOMET_VARS,PA_EP,PA_CELL,RH_EP,ROT_YAW,ROT_PITCH,ROUGHNESS_LENGTH,RESPONSE_TIME_SA,RESPONSE_TIME_GA_CO2,RESPONSE_TIME_GA_H2O,SW_IN_POT,SONIC_NR,SH_SINGLE,SLE_SINGLE,SET_SINGLE,SC_SINGLE,SH2O_SINGLE,SPECIFIC_HUMIDITY,SPECIFIC_HEAT_EVAP,SONIC_DIAG_NREX,SPECTRAL_CORRECTION_METHOD,TIMESTAMP_START,T_SONIC_NR,TAU_NR,TAU,TAU_RANDUNC_HF,TKE,TSTAR,T_SONIC,TA_EP,TDEW,T_SONIC_MEDIAN,T_SONIC_P25,T_SONIC_P75,T_SONIC_SIGMA,T_SONIC_SKW,T_SONIC_KUR,TAU_UNCORR,TAU_STAGE1,TAU_STAGE2,T_CELL,TAU_SCF,T_SONIC_NUM_SPIKES,T_SONIC_SPIKE_NREX,T_SONIC_ABSLIM_NREX,T_SONIC_VM97_TEST,T_SONIC_LGD,T_SONIC_KID,T_SONIC_ZCD,TAU_CORRDIFF,TAU_NSR,TAU_SS,T_SONIC_ITC,TAU_SS_TEST,T_SONIC_ITC_TEST,TAU_SSITC_TEST,TIMELAG_DETECTION_METHOD,U_UNROT,U,USTAR,U_MEDIAN,U_P25,U_P75,U_SIGMA,U_SKW,U_KUR,USTAR_UNCORR,U_NUM_SPIKES,U_SPIKE_NREX,U_ABSLIM_NREX,U_VM97_TEST,U_LGD,U_KID,U_ZCD,U_ITC,U_ITC_TEST,V_UNROT,V,VAPOR_DENSITY,VAPOR_PARTIAL_PRESSURE,VAPOR_PARTIAL_PRESSURE_SAT,VPD_EP,VAPOR_DRYAIR_RATIO,V_MEDIAN,V_P25,V_P75,V_SIGMA,V_SKW,V_KUR,V_NUM_SPIKES,V_SPIKE_NREX,V_ABSLIM_NREX,V_VM97_TEST,VM97_TLAG_HF,VM97_TLAG_SF,VM97_AOA_HF,VM97_NSHW_HF,V_LGD,V_KID,V_ZCD,VPATH_SA,VPATH_GA_CO2,VPATH_GA_H2O,WD_FILTER_NR,W_UNROT,W,WS,WS_MAX,WD,WD_SIGMA,W_MEDIAN,W_P25,W_P75,W_SIGMA,W_SKW,W_KUR,W_U_COV,W_T_SONIC_COV,W_CO2_MEAS_COV,W_H2O_MEAS_COV,W_T_SONIC_COV_IBROM,W_T_SONIC_COV_IBROM_N1626,W_T_SONIC_COV_IBROM_N0614,W_T_SONIC_COV_IBROM_N0277,W_T_SONIC_COV_IBROM_N0133,W_T_SONIC_COV_IBROM_N0065,W_T_SONIC_COV_IBROM_N0032,W_T_SONIC_COV_IBROM_N0016,W_T_SONIC_COV_IBROM_N0008,W_T_SONIC_COV_IBROM_N0004,W_NUM_SPIKES,WD_FILTER_NREX,W_SPIKE_NREX,W_ABSLIM_NREX,W_VM97_TEST,W_LGD,W_KID,W_ZCD,W_ITC,W_ITC_TEST,WBOOST_APPLIED,WPL_APPLIED,ZL,ZL_UNCORR,BADM_INSTPAIR_EASTWARD_SEP_GA_CH4,BADM_INSTPAIR_EASTWARD_SEP_GA_N2O,BADM_INSTPAIR_HEIGHT_S

# Sort column order

In [8]:
df = df.reindex(sorted(df.columns), axis=1)  # Sort column order
df

,AIR_CP,AIR_DENSITY,AIR_MV,AIR_RHO_CP,AOA_METHOD,AXES_ROTATION_METHOD,BADM_HEIGHTC,BADM_INSTPAIR_EASTWARD_SEP_GA_CH4,BADM_INSTPAIR_EASTWARD_SEP_GA_CO2,BADM_INSTPAIR_EASTWARD_SEP_GA_H2O,BADM_INSTPAIR_EASTWARD_SEP_GA_N2O,BADM_INSTPAIR_HEIGHT_SEP_GA_CH4,BADM_INSTPAIR_HEIGHT_SEP_GA_CO2,BADM_INSTPAIR_HEIGHT_SEP_GA_H2O,BADM_INSTPAIR_HEIGHT_SEP_GA_N2O,BADM_INSTPAIR_NORTHWARD_SEP_GA_CH4,BADM_INSTPAIR_NORTHWARD_SEP_GA_CO2,BADM_INSTPAIR_NORTHWARD_SEP_GA_H2O,BADM_INSTPAIR_NORTHWARD_SEP_GA_N2O,BADM_INST_AVERAGING_INT,BADM_INST_GA_CP_TUBE_FLOW_RATE_GA_CH4,BADM_INST_GA_CP_TUBE_FLOW_RATE_GA_H2O,BADM_INST_GA_CP_TUBE_FLOW_RATE_GA_N2O,BADM_INST_GA_CP_TUBE_IN_DIAM_GA_CH4,BADM_INST_GA_CP_TUBE_IN_DIAM_GA_H2O,BADM_INST_GA_CP_TUBE_IN_DIAM_GA_N2O,BADM_INST_GA_CP_TUBE_LENGTH_GA_CH4,BADM_INST_GA_CP_TUBE_LENGTH_GA_H2O,BADM_INST_GA_CP_TUBE_LENGTH_GA_N2O,BADM_INST_HEIGHT_SA,BADM_INST_SAMPLING_INT,BADM_LOCATION_ELEV,BADM_LOCATION_LAT,BADM_LOCATION_LONG,BADM_SA_OFFSET_NORTH,BOWEN,BURBA_METHOD,CH4,CH4_ABSLIM_NREX,CH4_DIAG_NREX,CH4_KID,CH4_LGD,CH4_MEAS_KUR,CH4_MEAS_MEDIAN,CH4_MEAS_N2O_MEAS_COV,CH4_MEAS_P25,CH4_MEAS_P75,CH4_MEAS_SIGMA,CH4_MEAS_SKW,CH4_MEAS_TYPE,CH4_MIXING_RATIO,CH4_MOLAR_DENSITY,CH4_NR,CH4_NUM_SPIKES,CH4_SPIKE_NREX,CH4_TLAG_ACTUAL,CH4_TLAG_MAX,CH4_TLAG_MIN,CH4_TLAG_NOMINAL,CH4_TLAG_USED,CH4_VM97_TEST,CH4_ZCD,CO2,CO2_ABSLIM_NREX,CO2_DIAG_NREX,CO2_KID,CO2_LGD,CO2_MEAS_H2O_MEAS_COV,CO2_MEAS_KUR,CO2_MEAS_MEDIAN,CO2_MEAS_P25,CO2_MEAS_P75,CO2_MEAS_SIGMA,CO2_MEAS_SKW,CO2_MEAS_TYPE,CO2_MIXING_RATIO,CO2_MOLAR_DENSITY,CO2_NR,CO2_NUM_SPIKES,CO2_SPIKE_NREX,CO2_TLAG_ACTUAL,CO2_TLAG_MAX,CO2_TLAG_MIN,CO2_TLAG_NOMINAL,CO2_TLAG_USED,CO2_VM97_TEST,CO2_ZCD,CUSTOM_AGC_MEAN,CUSTOM_AIR_P_MEAN,CUSTOM_CH4_MEAN,CUSTOM_COOLER_V_MEAN,CUSTOM_DATA_SIZE_IRGA75_MEAN,CUSTOM_DATA_SIZE_LGR_MEAN,CUSTOM_DATA_SIZE_QCL_MEAN,CUSTOM_DUMMY_MEAN,CUSTOM_FAST_T_MEAN,CUSTOM_FILTER_NR,CUSTOM_FILTER_NREX,CUSTOM_FIT_FLAG_LGR_MEAN,CUSTOM_GA_DIAG_CODE_IRGA75_MEAN,CUSTOM_MIRROR_RINGDOWNTIME_LGR_MEAN,CUSTOM_N2O_MEAN,CUSTOM_STATUS_CODE_IRGA75_MEAN,CUSTOM_STATUS_CODE_LGR_MEAN,CUSTOM_STATUS_CODE_QCL_MEAN,CUSTOM_STATUS_QCL_MEAN,CUSTOM_STATUS_WORD_QCL_MEAN,CUSTOM_T_UNKNOWN_MEAN,CUSTOM_VICI_MEAN,DENTRENDING_TIME_CONSTANT,DETRENDING_METHOD,DISPLACEMENT_HEIGHT,DOY_END,DOY_START,DRYAIR_DENSITY,DRYAIR_MV,DRYAIR_PARTIAL_PRESSURE,ET,ET_CORRDIFF,ET_RANDUNC_HF,ET_SCF,ET_SSITC_TEST,ET_STAGE1,ET_STAGE2,ET_UNCORR,EXPECT_NR,FC,FCH4,FCH4_CORRDIFF,FCH4_NR,FCH4_NSR,FCH4_RANDUNC_HF,FCH4_SCF,FCH4_SS,FCH4_SSITC_TEST,FCH4_SS_TEST,FCH4_STAGE1,FCH4_STAGE2,FCH4_UNCORR,FCH4_VADV,FC_CORRDIFF,FC_NR,FC_NSR,FC_RANDUNC_HF,FC_SCF,FC_SS,FC_SSITC_TEST,FC_SS_TEST,FC_STAGE1,FC_STAGE2,FC_UNCORR,FC_VADV,FETCH_10,FETCH_30,FETCH_50,FETCH_70,FETCH_80,FETCH_90,FETCH_MAX,FETCH_OFFSET,FH2O,FH2O_CORRDIFF,FH2O_NSR,FH2O_RANDUNC_HF,FH2O_SCF,FH2O_SS,FH2O_SSITC_TEST,FH2O_SS_TEST,FH2O_STAGE1,FH2O_STAGE2,FH2O_UNCORR,FH2O_VADV,FILE_NR,FILE_TIME_DURATION,FLAG_PA_GF1_0.9_1_ISFILLED,FLAG_PPFD_IN_T1_2_2_ISFILLED,FLAG_PREC_RAIN_TOT_GF1_0.5_1_FLUXNET_ISFILLED,FLAG_PREC_RAIN_TOT_GF1_0.5_1_ISFILLED,FLAG_RH_T1_2_1_ISFILLED,FLAG_SWC_GF1_0.15_1_gfXG_ISFILLED,FLAG_SW_IN_T1_2_1_ISFILLED,FLAG_TA_T1_2_1_ISFILLED,FLAG_TS_GF1_0.04_1_gfXG_ISFILLED,FLAG_TS_GF1_0.15_1_gfXG_ISFILLED,FLAG_TS_GF1_0.4_1_gfXG_ISFILLED,FLAG_VPD_T1_2_1_ISFILLED,FN2O,FN2O_CORRDIFF,FN2O_NR,FN2O_NSR,FN2O_RANDUNC_HF,FN2O_SCF,FN2O_SS,FN2O_SSITC_TEST,FN2O_SS_TEST,FN2O_STAGE1,FN2O_STAGE2,FN2O_UNCORR,FN2O_VADV,FOOTPRINT_MODEL,H,H2O,H2O_ABSLIM_NREX,H2O_DIAG_NREX,H2O_KID,H2O_LGD,H2O_MEAS_CH4_MEAS_COV,H2O_MEAS_KUR,H2O_MEAS_MEDIAN,H2O_MEAS_N2O_MEAS_COV,H2O_MEAS_P25,H2O_MEAS_P75,H2O_MEAS_SIGMA,H2O_MEAS_SKW,H2O_MEAS_TYPE,H2O_MIXING_RATIO,H2O_MOLAR_DENSITY,H2O_NR,H2O_NUM_SPIKES,H2O_SPIKE_NREX,H2O_TLAG_ACTUAL,H2O_TLAG_MAX,H2O_TLAG_MIN,H2O_TLAG_NOMINAL,H2O_TLAG_USED,H2O_VM97_TEST,H2O_ZCD,HPATH_GA_CH4,HPATH_GA_CO2,HPATH_GA_H2O,HPATH_GA_N2O,HPATH_SA,H_CORRDIFF,H_NR,H_NSR,H_RANDUNC_HF,H_SCF,H_SS,H_SSITC_TEST,H_SS_TEST,H_STAGE1,H_STAGE2,H_UNCORR,INST_LI7200_AGC_OR_RSSI,INST_LI7500_AGC_OR_RSSI,LE,LE_CORRDIFF,LE_NR

# Save to file

In [9]:
OUTNAME = "33.1_CH-CHA_IRGA+QCL+LGR+M10+MGMT_Level-1_eddypro_fluxnet_2005-2024"
filepath = save_parquet(filename=OUTNAME, data=df)
df.to_csv(f"{OUTNAME}.csv")

Saved file 33.1_CH-CHA_IRGA+QCL+LGR+M10+MGMT_Level-1_eddypro_fluxnet_2005-2024.parquet (6.703 seconds).


# **End of notebook**.

In [10]:
dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Finished. {dt_string}")

Finished. 2025-01-25 12:59:04
